In [0]:
import gym
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as n
import torch.nn.functional as f

In [0]:
!pip install ptan

In [0]:
env=gym.make("CartPole-v0")

In [0]:
import ptan
import torch.optim as op

In [0]:
obsdim=env.observation_space.shape[0]
nactions=env.action_space.n

In [0]:
class model(n.Module):
  def __init__(self,obsdim,nactions):
    super().__init__()
    self.fc1=n.Linear(obsdim,128)
    self.fc2=n.Linear(128,nactions)
  
  def forward(self,x):
    x=x.float()
    out=f.relu(self.fc1(x))
    return self.fc2(out)

# en  

In [0]:
net=model(obsdim,nactions)

In [0]:
# Test epsilongreedyactionselector
a=torch.randn((8,4))
agent(a)￼￼
￼
￼

NameError: ignored

In [0]:
expsource=ptan.experience.ExperienceSourceFirstLast(env,agent,gamma=0.99,steps_count=2) 
# s1 a1 r1 s2 a2 r2 s3 a3 r3 s4     will give  s1 a1  r1 +gamma*r2  s3

NameError: ignored

In [0]:
it=iter(expsource)

In [0]:
next(it)

In [0]:
count=0
for i in expsource:
  print(i)
  count+=1
  if count==6:
    break

In [0]:
buffer=ptan.experience.ExperienceReplayBuffer(expsource,buffer_size=10000)

NameError: ignored

In [0]:
optimizer=op.Adam(net.parameters(),lr=0.001,)

In [0]:
frameidx=0

In [0]:
buffer.populate(5)

NameError: ignored

In [0]:
buffer.buffer     # gives the s a r s values and stores them in the buffer

NameError: ignored

In [0]:
tgtnet=ptan.agent.TargetNet(net)

In [0]:
class epsilontracker:
  def __init__(self,selector,epsstart,epsfinal,epsframes):
    self.selector=selector
    self.epsstart=epsstart
    self.epsfinal=epsfinal
    self.epsframes=epsframes
    self.frame(0)
  
  def frame(self,frame):
    self.selector.epsilon=max(self.epsfinal,self.epsstart-frame/self.epsframes)

In [0]:
epsstart=1.0

In [0]:
actionselector=ptan.actions.EpsilonGreedyActionSelector(epsstart)  # epsilongreedyactionselector is given qvalues
epsilontracker=epsilontracker(actionselector,1,0.1,100000)

In [0]:
agent=ptan.agent.DQNAgent(net,actionselector)

In [0]:
tgtnet=ptan.agent.TargetNet(net)

In [0]:
expsource=ptan.experience.ExperienceSourceFirstLast(env,agent,gamma=0.99,steps_count=1)
buffer=ptan.experience.ExperienceReplayBuffer(expsource,10000)

In [0]:
class rewardtracker:
  def __init__(self,stopreward):
    self.stopreward=stopreward
    self.totalreward=[]
  
  def reward(self,reward1,frame,epsilon=None):
    self.totalreward.append(reward1)
    meanreward=np.mean(self.totalreward[-100:])

    if meanreward>self.stopreward:
      print("Solved")
      return True  
    return False

In [0]:
stopreward=100
rt=rewardtracker(stopreward)

In [0]:
def unpackbatch(batch):
  states,actions,rewards,dones,last_states=[],[],[],[],[]
  for exp in batch:
    state=np.array(exp.state,copy=False)
    states.append(state)
    actions.append(exp.action)
    rewards.append(exp.reward)
    dones.append(exp.last_state is None)
    if exp.last_state is None:
      last_states.append(state)
    else:
      last_states.append(exp.last_state)
    return np.array(states,copy=False),np.array(actions,copy=False),np.array(rewards,dtype=np.float32), \
           np.array(dones,dtype=np.uint8) , np.array(last_states,copy=False)

In [0]:
#Experiment
dones=[True,False,True]
torch.ByteTensor(np.array(dones,dtype=np.uint8))

tensor([1, 0, 1], dtype=torch.uint8)

In [0]:
def calclossdqn(batch,net,tgtnet,gamma,device='cpu'):
  states,actions,rewards,dones,laststate=unpackbatch(batch)
  # states bs x statedims  # actions=bs x actionsdims
  # laststate= bs x statedims

  statesv=torch.tensor(states).to(device)
  nextstatesv=torch.tensor(laststate).to(device)
  actionsv=torch.tensor(actions).to(device)
  rewardsv=torch.tensor(rewards).to(device)
  donemask=torch.ByteTensor(dones).to(device)

  stateactionvalues=net(statesv).gather(1,actionsv.unsqueeze(-1)).squeeze(-1)
  nextstatevalues=tgtnet(nextstatesv).max(1)[0] # actionvalues from next state
  nextstatevalues[donemask]=0.0   #   for terminal state experience we want next state action values to be zero
  expectedstate_actval=nextstatevalues.detach()*gamma + rewardsv
  return n.MSELoss()(stateactionvalues,expectedstate_actval)

In [0]:
# Experiment gather 
a=torch.randint(0,10,(4,3))

In [0]:
actions=torch.tensor([1,2,0,1])  #   of shape (4,)

In [0]:
actions.unsqueeze(-1).shape

torch.Size([4, 1])

In [0]:
a.gather(1,actions.unsqueeze(-1))

tensor([[3],
        [8],
        [9],
        [8]])

In [0]:
a

tensor([[0, 3, 6],
        [6, 4, 8],
        [9, 4, 0],
        [3, 8, 7]])

In [0]:
gamma=0.99
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
targetnetsync=500

In [0]:
i=0
for e in expsource:
  print(e.reward,"---",e.last_state,"---",expsource.pop_total_rewards())
   # expsource will gives an empty list if terminal state is not achieved
  i+=1                                   # else if terminal state is reached then expsource will give reward
  if i==50:
    break

1.0 --- [-0.00223253 -0.22156729 -0.01998906  0.2964656 ] --- []
1.0 --- [-0.00666388 -0.41639866 -0.01405975  0.58277789] --- []
1.0 --- [-0.01499185 -0.22108258 -0.00240419  0.28569929] --- []
1.0 --- [-0.0194135  -0.02592643  0.0033098  -0.00774093] --- []
1.0 --- [-0.01993203 -0.22109569  0.00315498  0.28598444] --- []
1.0 --- [-0.02435394 -0.02601887  0.00887467 -0.00570176] --- []
1.0 --- [-0.02487432  0.16897468  0.00876063 -0.29557146] --- []
1.0 --- [-0.02149483  0.36397065  0.0028492  -0.5854786 ] --- []
1.0 --- [-0.01421541  0.16880891 -0.00886037 -0.29189951] --- []
1.0 --- [-0.01083924  0.36405607 -0.01469836 -0.58736366] --- []
1.0 --- [-0.00355811  0.55938075 -0.02644563 -0.8846402 ] --- []
1.0 --- [ 0.0076295   0.7548516  -0.04413844 -1.18551807] --- []
1.0 --- [ 0.02272653  0.56032905 -0.0678488  -0.90699091] --- []
1.0 --- [ 0.03393311  0.36618805 -0.08598862 -0.63638168] --- []
1.0 --- [ 0.04125687  0.56239731 -0.09871625 -0.95485767] --- []
1.0 --- [ 0.05250482  0.3

In [0]:
frameidx=0
while True:
  frameidx+=1
  buffer.populate(1)
  epsilontracker.frame(frameidx)
  newrewards=expsource.pop_total_rewards()